# EJERCICIOS

In [23]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_date, year, monotonically_increasing_id,  avg, min, coalesce, col, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, FloatType

spark = SparkSession.builder.master("local[*]").appName("pyspark_rdd").getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## EJERCICIO 0
En un documento word haz una lista de las diferentes operaciones con una breve descripción de lo que hace y un ejemplo de como se utiliza

## EJERCICIO 1
Realiza las siguientes operaciones:
* Importa el csv de "data/WorldCupPlayers.csv" (que deduzca el esquema)
* Visualiza los datos
* ¿Que tipo de datos contiene cada variable?
* ¿Cuantos registros hay?
* Obtén los principales estadísticos de Position
* Selecciona y muestra los "Team initials" diferentes que hay ¿Cuántos hay?
* ¿Cuantos partidos con el ID de 1096 ha habido?
* Muestra los datos donde la posicion haya sido C y el evento sea G40
* Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20

In [7]:
dfPlayers = (
    spark.read.option("sep", ",")
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("Datos/WorldCupPlayers.csv")
)

In [6]:
dfPlayers.show(1)

+-------+-------+-------------+-------------------+-------+------------+-----------+--------+-----+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|Player Name|Position|Event|
+-------+-------+-------------+-------------------+-------+------------+-----------+--------+-----+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|Alex THEPOT|      GK| NULL|
+-------+-------+-------------+-------------------+-------+------------+-----------+--------+-----+
only showing top 1 row



In [8]:
dfPlayers.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [9]:
dfPlayers.count()

37784

In [12]:
dfPlayers.describe("position").show()

+-------+--------+
|summary|position|
+-------+--------+
|  count|    4143|
|   mean|    NULL|
| stddev|    NULL|
|    min|       C|
|    max|     GKC|
+-------+--------+



In [13]:
dfPlayers.select("Team Initials").distinct().show()

+-------------+
|Team Initials|
+-------------+
|          POL|
|          JAM|
|          BRA|
|          CUB|
|          FRA|
|          ALG|
|          BOL|
|          RSA|
|          ITA|
|          UKR|
|          CMR|
|          SCG|
|          GHA|
|          SEN|
|          TOG|
|          TRI|
|          TCH|
|          AUS|
|          MEX|
|          PAR|
+-------------+
only showing top 20 rows



In [16]:
dfPlayers.filter(dfPlayers.MatchID == 1096).count()

33

In [17]:
dfPlayers.filter((dfPlayers.Event == "G40") & (dfPlayers.Position == "C")).show()

+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
|RoundID|MatchID|Team Initials|Coach Name|Line-up|Shirt Number|Player Name|Position|Event|
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+



In [18]:
dfPlayers.createOrReplaceTempView("players_table")
spark.sql("SELECT * FROM players_table WHERE MatchID >= 20").show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

## EJERCICIO 2

A partir del archivo nombres.json, crea un DataFrame y realiza las siguientes operaciones:

1. Crea una nueva columna (columna Mayor30) que indique si la persona es mayor de 30 años.
2. Crea una nueva columna (columna FaltanJubilacion) que calcule cuantos años le faltan para jubilarse (supongamos que se jubila a los 67 años)
3. Crea una nueva columna (columna Apellidos) que contenga XYZ (puedes utilizar la función lit)
4. Elimina las columna Mayor30 y Apellidos.
5. Crea una nueva columna (columna AnyoNac) con el año de nacimiento de cada persona (puedes utilizar la función current_date).
6. Añade un id incremental para cada fila (campo Id) y haz que al hacer un show se vea en primer lugar (puedes utilizar la función monotonically_increasing_id) seguidos del Nombre, Edad, AnyoNac, FaltaJubilacion y Ciudad

Al realizar los seis pasos, el resultado del DataFrame será similar a :
``````
+---+-------+----+-------+----------------+--------+
| Id|Nombre |Edad|AnyoNac|FaltanJubilacion|  Ciudad|
+---+-------+----+-------+----------------+--------+
|  0|  Aitor|  45|   1977|              22|   Elche|
|  1| Marina|  14|   2008|              53|Alicante|
|  2|  Laura|  19|   2003|              48|   Elche|
|  3|  Sonia|  45|   1977|              22|    Aspe|
|  4|  Pedro|null|   null|            null|   Elche|
+---+-------+----+-------+----------------+--------+
``````

In [24]:
dfNombre = spark.read.json("Datos/nombres.json")
dfNombre.show(1)

+------+----+------+
|Ciudad|Edad|Nombre|
+------+----+------+
| Elche|  45| Aitor|
+------+----+------+
only showing top 1 row



In [25]:
dfNombre = dfNombre.withColumn("mayor30", when(col("Edad") > 30, "Y").otherwise("N"))
dfNombre.show(1)

+------+----+------+-------+
|Ciudad|Edad|Nombre|mayor30|
+------+----+------+-------+
| Elche|  45| Aitor|      Y|
+------+----+------+-------+
only showing top 1 row



In [26]:
dfNombre = dfNombre.withColumn("FaltaJubilacion", 67 - col("Edad"))
dfNombre.show(1)

+------+----+------+-------+---------------+
|Ciudad|Edad|Nombre|mayor30|FaltaJubilacion|
+------+----+------+-------+---------------+
| Elche|  45| Aitor|      Y|             22|
+------+----+------+-------+---------------+
only showing top 1 row



In [27]:
dfNombre = dfNombre.withColumn("Apellidos", lit("XYZ"))
dfNombre.show(1)

+------+----+------+-------+---------------+---------+
|Ciudad|Edad|Nombre|mayor30|FaltaJubilacion|Apellidos|
+------+----+------+-------+---------------+---------+
| Elche|  45| Aitor|      Y|             22|      XYZ|
+------+----+------+-------+---------------+---------+
only showing top 1 row



In [28]:
dfNombre = dfNombre.drop("Apellidos", "mayor30")
dfNombre.show(1)

+------+----+------+---------------+
|Ciudad|Edad|Nombre|FaltaJubilacion|
+------+----+------+---------------+
| Elche|  45| Aitor|             22|
+------+----+------+---------------+
only showing top 1 row



In [29]:
dfNombre = dfNombre.withColumn("AnyoNac", year(current_date()) - col("Edad"))
dfNombre.show(1)

+------+----+------+---------------+-------+
|Ciudad|Edad|Nombre|FaltaJubilacion|AnyoNac|
+------+----+------+---------------+-------+
| Elche|  45| Aitor|             22|   1980|
+------+----+------+---------------+-------+
only showing top 1 row



In [30]:
dfNombre = dfNombre.withColumn("Id", monotonically_increasing_id())
dfNombre = dfNombre.select("Id", "Nombre", "Edad", "AnyoNac", "FaltaJubilacion", "Ciudad")
dfNombre.show()

+---+------+----+-------+---------------+--------+
| Id|Nombre|Edad|AnyoNac|FaltaJubilacion|  Ciudad|
+---+------+----+-------+---------------+--------+
|  0| Aitor|  45|   1980|             22|   Elche|
|  1|Marina|  14|   2011|             53|Alicante|
|  2| Laura|  19|   2006|             48|   Elche|
|  3| Sonia|  45|   1980|             22|    Aspe|
|  4| Pedro|NULL|   NULL|           NULL|   Elche|
+---+------+----+-------+---------------+--------+



## EJERCICIO 3

A partir del archivo VentasNulos.csv:

1. Elimina las filas que tengan al menos 4 nulos.

2. Con las filas restantes, sustituye:

    * Los nombres nulos por Empleado
    * Las ventas nulas por la media de las ventas de los compañeros (redondeado a entero).
    ``````
        media = df.groupBy().avg('Ventas')
    ``````
    * Los euros nulos por el valor del compañero que menos € ha ganado. (tras agrupar, puedes usar la función min)
    * La ciudad nula por C.V. y el identificador nulo por XYZ

In [10]:
dfNull = (
    spark.read.option("sep", ",")
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("Datos/VentasNulos.csv")
)
dfNull.show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| NULL|       NULL|         NULL|
|  Juan|  NULL| NULL|       NULL|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  NULL|  300| Torrellano|         NULL|
|Marina|     3|  350|       Aspe|          V55|
|  NULL|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
|  NULL|  NULL| NULL|       NULL|         NULL|
| Jorge|     8| NULL|       NULL|          T19|
+------+------+-----+-----------+-------------+



In [8]:
dfNull = dfNull.filter(sum(col(c).isNull().cast("int") for c in dfNull.columns) < 4)
dfNull.show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| NULL|       NULL|         NULL|
|  Juan|  NULL| NULL|       NULL|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  NULL|  300| Torrellano|         NULL|
|Marina|     3|  350|       Aspe|          V55|
|  NULL|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
| Jorge|     8| NULL|       NULL|          T19|
+------+------+-----+-----------+-------------+



In [11]:
dfNull = dfNull.na.fill("Empleado",subset="Nombre")
dfNull.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| NULL|       NULL|         NULL|
|    Juan|  NULL| NULL|       NULL|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|  NULL|  300| Torrellano|         NULL|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|  NULL| NULL|       NULL|         NULL|
|   Jorge|     8| NULL|       NULL|          T19|
+--------+------+-----+-----------+-------------+



In [13]:
media = dfNull.select(avg("Ventas")).collect()[0][0]
dfNull = dfNull.na.fill({"Ventas": media})
dfNull.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| NULL|       NULL|         NULL|
|    Juan|     6| NULL|       NULL|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|         NULL|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6| NULL|       NULL|         NULL|
|   Jorge|     8| NULL|       NULL|          T19|
+--------+------+-----+-----------+-------------+



In [14]:
min_euros = dfNull.select(min("Euros")).collect()[0][0]
dfNull = dfNull.na.fill({"Euros": min_euros})
dfNull.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8|   30|       NULL|         NULL|
|    Juan|     6|   30|       NULL|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|         NULL|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6|   30|       NULL|         NULL|
|   Jorge|     8|   30|       NULL|          T19|
+--------+------+-----+-----------+-------------+



In [15]:
dfNull = dfNull.na.fill({"Ciudad":"C.V."})
dfNull = dfNull.na.fill({"Identificador": "XYZ"})
dfNull.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8|   30|       C.V.|          XYZ|
|    Juan|     6|   30|       C.V.|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|          XYZ|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6|   30|       C.V.|          XYZ|
|   Jorge|     8|   30|       C.V.|          T19|
+--------+------+-----+-----------+-------------+



## EJERCICIO 4

 A partir del archivo movies.tsv, crea una esquema de forma declarativa con los campos:

* interprete de tipo string
* pelicula de tipo string
* anyo de tipo int

Cada fila del fichero implica que el actor/actriz ha trabajado en dicha película en el año indicado.
1. Una vez creado el esquema, carga los datos en un DataFrame.

A continuación, mediante el DataFrame API:

2. Muestra las películas en las que ha trabajado Murphy, Eddie (I).
3. Muestra los intérpretes que aparecen tanto en Superman como en Superman II.

In [ ]:
esquema = StructType(
    [
        StructField("interprete", StringType(), False),
        StructField("pelicula", StringType(), False),
        StructField("anyo", IntegerType(), False),
    ]
)
dfMovies = spark.read.option("sep", "\t").schema(esquema).csv("Datos/movies.tsv")
dfMovies.show()

+-----------------+--------------------+----+
|       interprete|            pelicula|anyo|
+-----------------+--------------------+----+
|McClure, Marc (I)|       Freaky Friday|2003|
|McClure, Marc (I)|        Coach Carter|2005|
|McClure, Marc (I)|         Superman II|1980|
|McClure, Marc (I)|           Apollo 13|1995|
|McClure, Marc (I)|            Superman|1978|
|McClure, Marc (I)|  Back to the Future|1985|
|McClure, Marc (I)|Back to the Futur...|1990|
|Cooper, Chris (I)|  Me, Myself & Irene|2000|
|Cooper, Chris (I)|         October Sky|1999|
|Cooper, Chris (I)|              Capote|2005|
|Cooper, Chris (I)|The Bourne Supremacy|2004|
|Cooper, Chris (I)|         The Patriot|2000|
|Cooper, Chris (I)|            The Town|2010|
|Cooper, Chris (I)|          Seabiscuit|2003|
|Cooper, Chris (I)|      A Time to Kill|1996|
|Cooper, Chris (I)|Where the Wild Th...|2009|
|Cooper, Chris (I)|         The Muppets|2011|
|Cooper, Chris (I)|     American Beauty|1999|
|Cooper, Chris (I)|             Sy

In [20]:
dfMovies.filter(col("interprete") == "Murphy, Eddie (I)").select("pelicula").show()

+--------------------+
|            pelicula|
+--------------------+
|            Showtime|
|              Norbit|
|Hot Tub Time Machine|
|Nutty Professor I...|
|Beverly Hills Cop II|
|      Trading Places|
|      Daddy Day Care|
|      Dr. Dolittle 2|
| Shrek Forever After|
|   Beverly Hills Cop|
|               Shrek|
| The Haunted Mansion|
|   Coming to America|
|             Shrek 2|
|     Doctor Dolittle|
| The Nutty Professor|
|               Mulan|
|         Tower Heist|
|          Dreamgirls|
|           Bowfinger|
+--------------------+
only showing top 20 rows



In [21]:
dfSuperman1 = dfMovies.filter(col("pelicula") == "Superman").select("interprete")
dfSuperman2 = dfMovies.filter(col("pelicula") == "Superman II").select("interprete")
dfSuperman1.intersect(dfSuperman2).show()


+------------------+
|        interprete|
+------------------+
|  O'Halloran, Jack|
|   Tucker, Burnell|
|  Hollis, John (I)|
|       Beatty, Ned|
|    Stamp, Terence|
|Ratzenberger, John|
|     Hackman, Gene|
|    Fielder, Harry|
|  Perrine, Valerie|
| McClure, Marc (I)|
|   Donner, Richard|
+------------------+



## EJERCICIO 5
Realiza las siguientes operaciones:
* Carga el dataset de “data/stocks_price_final.csv”, con el esquema correcto de datos (tienes que crear tu el schema").
* Renombra la variable market.cap a market
* Elimina la variable market
* Muestra las filas donde el valor de "open" es nulo.
* Elimina las filas donde el valor de "open" es nulo.
* Para comprobar el punto anterior vuelve a mostrar las filas donde el valor de "open" es nulo.


In [34]:
esquema = StructType([
    StructField("", IntegerType(), False),
    StructField("symbol", StringType(), False),
    StructField("date", StringType(), False),
    StructField("open", FloatType(), False),
    StructField("high", FloatType(), False),
    StructField("low", FloatType(), False),
    StructField("close", FloatType(), False),
    StructField("volume", IntegerType(), False),
    StructField("adjusted", FloatType(), False),
    StructField("market.cap", StringType(), False),
    StructField("sector", StringType(), False),
    StructField("industry", StringType(), False),
    StructField("exchange", StringType(), False)
])
dfStock = spark.read.option("header", True).schema(esquema).csv("Datos/stocks_price_final.csv")
dfStock.show()

+---+------+----------+-----+------+------+-----+-------+--------+----------+-------------+--------------------+--------+
|   |symbol|      date| open|  high|   low|close| volume|adjusted|market.cap|       sector|            industry|exchange|
+---+------+----------+-----+------+------+-----+-------+--------+----------+-------------+--------------------+--------+
|  1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  5|   TXG|2019-09-18|56.85| 62.27| 55.65| 62.0|1589600|    62.0|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  6|   TXG|2019-09-19|6

In [35]:
dfStock = dfStock.withColumnRenamed("market.cap", "market")
dfStock.show()

+---+------+----------+-----+------+------+-----+-------+--------+------+-------------+--------------------+--------+
|   |symbol|      date| open|  high|   low|close| volume|adjusted|market|       sector|            industry|exchange|
+---+------+----------+-----+------+------+-----+-------+--------+------+-------------+--------------------+--------+
|  1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|$9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|$9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|$9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|$9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  5|   TXG|2019-09-18|56.85| 62.27| 55.65| 62.0|1589600|    62.0|$9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  6|   TXG|2019-09-19|62.81|63.375| 61.03|61.12| 425200

In [36]:
dfStock = dfStock.drop("market")
dfStock.show()

+---+------+----------+-----+------+------+-----+-------+--------+-------------+--------------------+--------+
|   |symbol|      date| open|  high|   low|close| volume|adjusted|       sector|            industry|exchange|
+---+------+----------+-----+------+------+-----+-------+--------+-------------+--------------------+--------+
|  1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|Capital Goods|Biotechnology: La...|  NASDAQ|
|  2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|Capital Goods|Biotechnology: La...|  NASDAQ|
|  3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|Capital Goods|Biotechnology: La...|  NASDAQ|
|  4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|Capital Goods|Biotechnology: La...|  NASDAQ|
|  5|   TXG|2019-09-18|56.85| 62.27| 55.65| 62.0|1589600|    62.0|Capital Goods|Biotechnology: La...|  NASDAQ|
|  6|   TXG|2019-09-19|62.81|63.375| 61.03|61.12| 425200|   61.12|Capital Goods|Biotechnology: La...|  NASDAQ|
|

In [37]:
dfStock.filter(col("open").isNull()).show()


+----+------+----------+----+----+----+-----+------+--------+-------------+--------------------+--------+
|    |symbol|      date|open|high| low|close|volume|adjusted|       sector|            industry|exchange|
+----+------+----------+----+----+----+-----+------+--------+-------------+--------------------+--------+
|4378|  KRKR|2020-05-11|NULL|NULL|NULL| NULL|  NULL|    NULL|Miscellaneous|   Business Services|  NASDAQ|
|5747|  NMTR|2020-01-23|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5748|  NMTR|2020-01-24|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5749|  NMTR|2020-01-27|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5750|  NMTR|2020-01-28|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5751|  NMTR|2020-01-29|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5752|  NMTR|2020-01-30|NULL|NULL|NULL| NULL| 

In [38]:
dfStock = dfStock.filter(col("open").isNotNull())
dfStock.filter(col("open").isNull()).show()

+---+------+----+----+----+---+-----+------+--------+------+--------+--------+
|   |symbol|date|open|high|low|close|volume|adjusted|sector|industry|exchange|
+---+------+----+----+----+---+-----+------+--------+------+--------+--------+
+---+------+----+----+----+---+-----+------+--------+------+--------+--------+

